In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model

from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization, Input
from tensorflow.keras.applications import VGG16, VGG19

import os

In [ ]:
image_path=('/content/drive/MyDrive/Deep Learning/CNN/Skin cancer dataset.zip (Unzipped Files)/Skin cancer dataset')

In [ ]:
test_dir = ('/content/drive/MyDrive/Deep Learning/CNN/Skin cancer dataset.zip (Unzipped Files)/Skin cancer dataset/test')
test_benign = ('/content/drive/MyDrive/Deep Learning/CNN/Skin cancer dataset.zip (Unzipped Files)/Skin cancer dataset/test/benign')
test_malignant = ('/content/drive/MyDrive/Deep Learning/CNN/Skin cancer dataset.zip (Unzipped Files)/Skin cancer dataset/test/malignant')

train_dir = ('/content/drive/MyDrive/Deep Learning/CNN/Skin cancer dataset.zip (Unzipped Files)/Skin cancer dataset/train')
train_benign = ('/content/drive/MyDrive/Deep Learning/CNN/Skin cancer dataset.zip (Unzipped Files)/Skin cancer dataset/train/benign')
train_malignant = ('/content/drive/MyDrive/Deep Learning/CNN/Skin cancer dataset.zip (Unzipped Files)/Skin cancer dataset/train/malignant')

val_dir = ('/content/drive/MyDrive/Deep Learning/CNN/Skin cancer dataset.zip (Unzipped Files)/Skin cancer dataset/validation')
val_benign = ('/content/drive/MyDrive/Deep Learning/CNN/Skin cancer dataset.zip (Unzipped Files)/Skin cancer dataset/validation/benign')
val_malignant = ('/content/drive/MyDrive/Deep Learning/CNN/Skin cancer dataset.zip (Unzipped Files)/Skin cancer dataset/validation/malignant')

In [ ]:
num_test_benign = len(os.listdir(test_benign))
num_test_malignant = len(os.listdir(test_malignant))

num_traint_benign = len(os.listdir(train_benign))
num_train_malignant = len(os.listdir(train_malignant))

num_val_benign = len(os.listdir(val_benign))
num_val_malignant = len(os.listdir(val_malignant))

In [ ]:
num_traint_benign,num_train_malignant

(1140, 937)

In [ ]:
num_test_benign,num_test_malignant

(360, 300)

In [ ]:
num_val_benign,num_val_malignant

(300, 260)

In [ ]:
X,Y = 224,224

from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_gen_train = ImageDataGenerator(rescale  = 1./255,
                                    rotation_range=40)
train_data_gen = image_gen_train.flow_from_directory(batch_size=64, directory = train_dir, target_size = (224,224),class_mode = 'binary')

image_gen_valid = ImageDataGenerator(rescale  = 1./255,
                                    rotation_range=40)
valid_data_gen = image_gen_valid.flow_from_directory(batch_size=64, directory = val_dir,target_size = (224,224),class_mode = 'binary')

image_gen_test = ImageDataGenerator(rescale  = 1./255,
                                    rotation_range=40)
test_data_gen = image_gen_test.flow_from_directory(batch_size=64, directory = test_dir, target_size = (224,224),class_mode = 'binary')

Found 2077 images belonging to 2 classes.
Found 560 images belonging to 2 classes.
Found 660 images belonging to 2 classes.


In [ ]:
vgg16 = VGG16(input_shape = (224,224,3), include_top = False, weights = 'imagenet')

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
for layer in vgg16.layers:
  layer.trainabel = False


In [ ]:
flatten = Flatten()(vgg16.output)

dense = Dense(128, activation = 'relu')(flatten)

dense = Dense(64, activation = 'relu')(dense)

prediction = Dense(1, activation = 'sigmoid')(dense)

In [ ]:
model = Model(inputs= vgg16.input, outputs=prediction)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
Metrics = ['accuracy', tf.keras.metrics.Recall()]

In [ ]:
model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics = Metrics)

In [ ]:
history = model.fit_generator(train_data_gen, epochs = 2,validation_data = valid_data_gen)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """Entry point for launching an IPython kernel.


Epoch 1/2
33/33 [==============================] - 1925s 58s/step - loss: 1.7713 - accuracy: 0.5190 - recall: 0.3383 - val_loss: 0.6545 - val_accuracy: 0.5357 - val_recall: 0.0000e+00
Epoch 2/2
33/33 [==============================] - 44s 1s/step - loss: 0.6599 - accuracy: 0.5749 - recall: 0.5155 - val_loss: 0.5301 - val_accuracy: 0.7661 - val_recall: 0.9462
